In [ ]:
!pip install transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
import re
from datasets import Dataset
from evaluate import load
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    BartForConditionalGeneration, BartTokenizer,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
import torch


In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.4 MB/s eta 0:00:00


In [ ]:
import requests

base_url = "https://raw.githubusercontent.com/google-research-datasets/wiki-split/master/"
files = ["tune.tsv", "validation.tsv", "test.tsv"]

for file in files:
    url = base_url + file
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {file}")
    else:
        print(f"Failed to download {file}")


Downloaded tune.tsv
Downloaded validation.tsv
Downloaded test.tsv


In [ ]:
# Load datasets
train_df = pd.read_csv("tune.tsv", sep='\t', header=None, names=['original', 'split'])
val_df = pd.read_csv("validation.tsv", sep='\t', header=None, names=['original', 'split'])
test_df = pd.read_csv("test.tsv", sep='\t', header=None, names=['original', 'split'])

In [ ]:
sentence = train_df.head(1)

print(sentence.to_string())

                                                                                                                                                             original                                                                                                                                                                      split
0  ' ( 1990 ) was the second sequel to appear , though Hooper did not return to direct due to scheduling conflicts with another film , '' Spontaneous Combustion '' .  ' ( 1990 ) was the second sequel to appear . <::::> Though Hooper did not return to direct due to scheduling conflicts with another film , '' Spontaneous Combustion '' .


In [ ]:
train_sentences = train_df['original'].dropna().tolist()
val_sentences = val_df['original'].dropna().tolist()
test_sentences = test_df['original'].dropna().tolist()


In [ ]:
print(f"Length of train dataset: {len(train_sentences)}")
print(f"Length of validation dataset: {len(val_sentences)}")
print(f"Length of test dataset: {len(test_sentences)}")

Length of train dataset: 5000
Length of validation dataset: 5000
Length of test dataset: 5000


In [ ]:
train_sentences[0]

"' ( 1990 ) was the second sequel to appear , though Hooper did not return to direct due to scheduling conflicts with another film , '' Spontaneous Combustion '' ."

In [ ]:
keyboard_adj = {
    'a': 'qwsz', 'b': 'vghn', 'c': 'xdfv', 'd': 'ersfcx',
    'e': 'wsdr', 'f': 'rtgdvc', 'g': 'tyfhvb', 'h': 'yugjnb',
    'i': 'ujko', 'j': 'uikhmn', 'k': 'ijolm', 'l': 'kop',
    'm': 'njk', 'n': 'bhjm', 'o': 'iklp', 'p': 'ol',
    'q': 'wa', 'r': 'edft', 's': 'wedxza', 't': 'rfgy',
    'u': 'yhji', 'v': 'cfgb', 'w': 'qase', 'x': 'zsdc',
    'y': 'tghu', 'z': 'asx',
}

def introduce_typos(sentence, typo_prob=0.2):
    def typo(word):
        if (
            len(word) <= 3 or
            not word.isalpha() or
            word[0].isupper() or
            (len(word) > 1 and word[1].isupper()) or
            random.random() > typo_prob
        ):
            return word

        ops = ['delete', 'swap', 'replace', 'add', 'keyboard']
        op = random.choice(ops)
        i = random.randint(0, len(word) - 1)
        c = word[i].lower()

        if op == 'delete':
            return word[:i] + word[i+1:]
        elif op == 'swap' and i < len(word) - 1:
            return word[:i] + word[i+1] + word[i] + word[i+2:]
        elif op == 'replace':
            return word[:i] + random.choice('abcdefghijklmnopqrstuvwxyz') + word[i+1:]
        elif op == 'add':
            return word[:i] + random.choice('abcdefghijklmnopqrstuvwxyz') + word[i:]
        elif op == 'keyboard' and c in keyboard_adj:
            replacement = random.choice(keyboard_adj[c])
            return word[:i] + replacement + word[i+1:]
        return word

    return ' '.join([typo(w) for w in sentence.split()])


In [ ]:
def generate_dataset(sentences, typo_prob=0.2, clean_percent=0.15, n_augmented=2):
    random.seed(42)
    corrupted = []
    targets = []

    for sent in sentences:
        if random.random() < clean_percent:
            corrupted.append(sent)
            targets.append(sent)
        else:
            for _ in range(n_augmented):
                corrupted.append(introduce_typos(sent, typo_prob))
                targets.append(sent)

    return pd.DataFrame({'input': corrupted, 'target': targets})


train_data = generate_dataset(train_sentences, typo_prob=0.2, clean_percent=0.15)
val_data = generate_dataset(val_sentences, typo_prob=0.2, clean_percent=0.15)
test_data = generate_dataset(test_sentences, typo_prob=0.2, clean_percent=0.15)


In [ ]:
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)


In [ ]:
train_dataset[0] , val_dataset[0], test_dataset[0]

({'input': "' ( 1990 ) was the shcond seque to appear , though Hooper did not return to drect due to scheduling conflicts with another film , '' Spontaneous Combustion '' .",
  'target': "' ( 1990 ) was the second sequel to appear , though Hooper did not return to direct due to scheduling conflicts with another film , '' Spontaneous Combustion '' ."},
 {'input': "' Lauren Rose Crace ' , bhrn 25th of May 1986 in Birmingham , England , currentl aplays the aprt of '' Ronnie Mitchell 's '' long lost daughter Danielle Jones ( Amy ) in the soap opera '' EastEnders '' .",
  'target': "' Lauren Rose Crace ' , born 25th of May 1986 in Birmingham , England , currently plays the part of '' Ronnie Mitchell 's '' long lost daughter Danielle Jones ( Amy ) in the soap opera '' EastEnders '' ."},
 {'input': "' Bandolier - Budgie ' , a fhee iTunes app for iPad , iPhone and iPod touc , areleased in December 2011 , tells the story of the making of Bandolier in the band 's own words - including an extensi

In [ ]:
print(f"Length of train dataset: {len(train_dataset)}")
print(f"Length of validation dataset: {len(val_dataset)}")
print(f"Length of test dataset: {len(test_dataset)}")

Length of train dataset: 9286
Length of validation dataset: 9283
Length of test dataset: 9266


In [ ]:
test_dataset = test_dataset.select(range(1000))
print(f"Length of test dataset: {len(test_dataset)}")

Length of test dataset: 1000


In [ ]:
max_train_len = 0
for example in train_dataset:
    max_train_len = max(max_train_len, len(example['input']), len(example['target']))

max_val_len = 0
for example in val_dataset:
    max_val_len = max(max_val_len, len(example['input']), len(example['target']))

print(f"Max train length: {max_train_len}")
print(f"Max val length: {max_val_len}")


max_length = max(max_train_len, max_val_len)  # = 346


Max train length: 340
Max val length: 350


In [ ]:
def preprocess(tokenizer, dataset):
    def tokenize(example):
        model_inputs = tokenizer(example["input"], max_length=128, truncation=True, padding="max_length")
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(example["target"], max_length=128, truncation=True, padding="max_length")
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    return dataset.map(tokenize, batched=True)


In [ ]:
!pip install Levenshtein fuzzywuzzy

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from transformers import T5Tokenizer, T5ForConditionalGeneration
import numpy as np
from Levenshtein import distance
from nltk.translate.gleu_score import corpus_gleu
from fuzzywuzzy import fuzz


In [ ]:
import os
def train_model(model_name, tokenizer_cls, model_cls):

    tokenizer = tokenizer_cls.from_pretrained(model_name)
    model = model_cls.from_pretrained(model_name)

    tokenized_train = preprocess(tokenizer, train_dataset)
    tokenized_val = preprocess(tokenizer, val_dataset)

    training_args = Seq2SeqTrainingArguments(
        output_dir=f"./{model_name}-finetuned-spell",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        weight_decay=0.01,
        save_total_limit=2,
        num_train_epochs=3,
        predict_with_generate=True,
        logging_dir="./logs",
        fp16=True
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    try:
        model = model_cls.from_pretrained(f"./{model_name}-finetuned-spell")
        tokenizer = tokenizer_cls.from_pretrained(f"./{model_name}-finetuned-spell")
        # Load trainer state directly on instantiation.
        trainer = Seq2SeqTrainer(
            model=model,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_val,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
        )
        print("Loading saved model...")
        # Load trainer state from checkpoint-3750
        trainer.train(resume_from_checkpoint=os.path.join(f"./{model_name}-finetuned-spell/checkpoint-3750"))

        print("Returning")
        return trainer, tokenizer, model
    except Exception as e:
        print(f"Error loading saved model: {e}")
        pass # or handle the exception appropriately
    print("Training model...")
    trainer.train()

    # save the model, tokenizer and trainer
    model.save_pretrained(f"./{model_name}-finetuned-spell")
    tokenizer.save_pretrained(f"./{model_name}-finetuned-spell")
    trainer.save_state()


    return trainer, tokenizer, model


In [ ]:
bart_trainer, bart_tokenizer, bart_model = train_model("facebook/bart-base", BartTokenizer, BartForConditionalGeneration)


Map:   0%|          | 0/9286 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9283 [00:00<?, ? examples/s]

<ipython-input-64-cec50a8bf832>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
<ipython-input-64-cec50a8bf832>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Loading saved model...
Error loading saved model: Can't find a valid checkpoint at ./facebook/bart-base-finetuned-spell/checkpoint-3750
Training model...


Step,Training Loss
500,0.015800
1000,0.011800
1500,0.008400
2000,0.006900
2500,0.007400
3000,0.006800


In [ ]:
pip install python-Levenshtein

In [ ]:
import Levenshtein

def compute_levenshtein(predictions, references):
    distances = []
    for pred, ref in zip(predictions, references):
        dist = Levenshtein.distance(pred, ref)
        distances.append(dist)
    avg_distance = sum(distances) / len(distances)
    return {"avg_levenshtein_distance": avg_distance}


In [ ]:
from evaluate import load
import Levenshtein

cer = load("cer")
wer = load("wer")

def evaluate_model_metrics_batched(trainer, tokenizer, dataset, batch_size=32):
    inputs = dataset['input']
    targets = dataset['target']

    preds = []
    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i:i+batch_size]
        tokenized = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=128).to(trainer.model.device)
        output_ids = trainer.model.generate(**tokenized, max_length=128)
        batch_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        preds.extend(batch_preds)

    cer_score = cer.compute(predictions=preds, references=targets)
    wer_score = wer.compute(predictions=preds, references=targets)
    levenshtein_score = compute_levenshtein(preds, targets)

    return {
        "CER": cer_score,
        "WER": wer_score,
        "Levenshtein": levenshtein_score
    }, preds


bart_metrics, bart_preds = evaluate_model_metrics_batched(bart_trainer, bart_tokenizer, test_dataset)


In [ ]:
bart_metrics

{'CER': 0.03087198271607646,
 'WER': 0.03935651513824414,
 'Levenshtein': {'avg_levenshtein_distance': 5.63}}

In [ ]:
def correct_sentence(model, tokenizer, sentence):
    inputs = tokenizer(sentence, return_tensors="pt").to(model.device)
    output_ids = model.generate(**inputs)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


user_input = "Ths is a sentnce with spleling errors."
print("User Input:", user_input)
print("BART Corrected:", correct_sentence(bart_model, bart_tokenizer, user_input))


User Input: Ths is a sentnce with spleling errors.
BART Corrected: Ths is a problem with spelling errors.


In [ ]:
user_input = "I haate Inter Milan fery much"
print("User Input:", user_input)
print("BART Corrected:", correct_sentence(bart_model, bart_tokenizer, user_input))

User Input: I haate Inter Milan fery much
BART Corrected: I love Inter Milan very much


In [ ]:
examples = [
    "I havw a dream about technology.",
    "The quick brown fox jumped over the lazey dog.",
    "Pleas correct this entire sentence with many typos."
]

for s in examples:
    print(f"\nOriginal:  {s}")
    print(f"Corrected: {correct_sentence(bart_model, bart_tokenizer, s)}")



Original:  I havw a dream about technology.
Corrected: I have a dream about technology.

Original:  The quick brown fox jumped over the lazey dog.
Corrected: The quick brown fox jumped over the startled dog.

Original:  Pleas correct this entire sentence with many typos.
Corrected: Pleas correct this entire sentence with many typos.


In [ ]:
for i in range(5):
    idx = random.randint(0, len(test_dataset))
    print(f"\nOriginal:  {test_dataset[idx]['input']}")
    print(f"Corrected: {correct_sentence(bart_model, bart_tokenizer, test_dataset[idx]['input'])}")


Original:  An elbum of the same name was also rdleased following the song 's success , nhich was also quite successful as was certified '' Gold '' by the RIAA in early 1973 for selling over 500,000 copies .
Corrected: An album of the same name was also released following the song 's success , which was

Original:  A cash settlement at the expiry of the contract will fund the financial lsos incurred by any rise in the physical jet fuel , allowing the companies to etter measure future cash flows .
Corrected: A cash settlement at the expiry of the contract will fund the financial losses incurred by any

Original:  '' Flying Easy Loving Crazy '' is Toshinobu Kubota 's thirty - third single , featuring R&B singer Misia , released on March 26 , 2008 .
Corrected: '' Flying Easy Loving Crazy '' is Toshinobu Kubota 's thirty - third

Original:  Arriving in Bad Wiessee nround 6 - 7h when the SA , still in bed , wsre takeen by surprise , the SS stofrmed the hotel and Hitler personally placed Röh

In [ ]:
original_grammer = "I have 3 shrits"
corrected_sentence = correct_sentence(bart_model, bart_tokenizer, original_grammer)
print(f"Original:  {original_grammer}")
print(f"Corrected: {corrected_sentence}")

Original:  I have 3 shrits
Corrected: I have 3 kids
